In [1]:
import pandas as pd
import numpy as np
import gcsfs
import recordlinkage as rl
import matching as m
import helpers as h
%load_ext autoreload
%autoreload 2

In [2]:
from gcsfs.core import GCSFileSystem
gcs = GCSFileSystem('courseradataeng', token='/home/jupyter/.config/gcloud/application_default_credentials.json') 

In [16]:
with gcs.open('soleadify_sample_data/unified_matched_data/matches_simplified.csv') as f:
    matches_simplified = pd.read_csv(f, quotechar='"', escapechar='\\')

In order to check if there are any duplicates in this dataset, I will rely on the root_domain as the unique key

In [17]:
print('There are {:,} rows in the dataset'.format(matches_simplified.shape[0]))
print('There are {:,} unique entities in the dataset'.format(matches_simplified['root_domain_final'].nunique()))

There are 29,761 rows in the dataset
There are 26,350 unique entities in the dataset


In [18]:
matches_simplified.columns

Index(['category__facebook', 'category__google', 'category__website',
       'description__facebook', 'description__google', 'email__facebook',
       'index__facebook', 'index__google', 'index__website',
       'language__website', 'legal_name__website', 'page_type__facebook',
       'country_clean_final', 'region_clean_final', 'city_clean_final',
       'root_domain_final', 'zip_code_unified', 'phone_unified',
       'site_name_final', 'address_unified'],
      dtype='object')

In [115]:
levels_of_logic = {
    'level_1':{'country_clean_final':'value'
              ,'region_clean_final':'value'
              ,'city_clean_final':'value'
              ,'root_domain_final':'value'
              ,'site_name_final':'notna'
             }
    ,'level_2':{'country_clean_final':'value'
              ,'region_clean_final':'value'
              ,'city_clean_final':'isna'
              ,'root_domain_final':'value'
              ,'site_name_final':'notna'
             }
    ,'level_3':{'country_clean_final':'value'
              ,'region_clean_final':'isna'
              ,'city_clean_final':'isna'
              ,'root_domain_final':'value'
              ,'site_name_final':'notna'
             }
    ,'level_4':{'country_clean_final':'isna'
              ,'region_clean_final':'isna'
              ,'city_clean_final':'isna'
              ,'root_domain_final':'value'
              ,'site_name_final':'notna'
             }
    ,'level_5':{'country_clean_final':'isna'
              ,'region_clean_final':'isna'
              ,'city_clean_final':'isna'
              ,'root_domain_final':'value'
              ,'site_name_final':'isna'
             }
}

In [124]:
uniques = pd.DataFrame()
deduplicated = []
for logic_dict in levels_of_logic.values():
    blocking_columns = []
    similar_columns = []
    for column in logic_dict.keys():
        if logic_dict[column] == 'value':
            blocking_columns.append(column)
        elif logic_dict[column] == 'notna':
            similar_columns.append(column)

    combinations = h.get_combinations(matches_simplified, None,blocking_columns)

    for combination in combinations:
        filtering_dict = h.get_filtering_dict(logic_dict, combination)
        filtering_query = h.get_filtering_query(filtering_dict)
        dataset = h.filter_data(matches_simplified, None, filtering_query, None)
        if dataset.shape[0]>1:
            out=m.unify_rows(dataset, blocking_columns)
            deduplicated.append(out)
        else:
            uniques = pd.concat([uniques, dataset])

In [127]:
uniques_ord = uniques.reindex(sorted(uniques.columns), axis=1)
deduplicated_df = pd.DataFrame(deduplicated)
deduplicated_df_ord = deduplicated_df.reindex(sorted(deduplicated_df.columns), axis=1)
final = pd.concat([uniques_ord, deduplicated_df_ord])

In [130]:
final['index__website'] = final['index__website'].astype('Int64')
final['index__google'] = final['index__google'].astype('Int64')
final['index__facebook'] = final['index__facebook'].astype('Int64')

In [132]:
with gcs.open('soleadify_sample_data/unified_and_unmatched_data/unified_unique.csv','w') as f:
    final.to_csv(f, index=False, quotechar='"', escapechar='\\')

In [137]:
objs = gcs.ls('soleadify_sample_data/clean_data')
for path in objs:
    if path[-4:] == '.csv':
        fileName = path.split('/')[-1]
        origin = fileName.split('_')[0]
        print(origin)
        with gcs.open(path) as f:
            if origin in ['facebook','google']:
                dataset = pd.read_csv(f, quotechar='"', escapechar='\\')
            else:
                dataset = pd.read_csv(f)
        umatched = dataset.loc[~dataset.index.isin(final['index__'+origin])]
        with gcs.open('soleadify_sample_data/unified_and_unmatched_data/'+origin+'_unmatched.csv','w') as f:
            umatched.to_csv(f, index=False, quotechar='"', escapechar='\\')

facebook
google
website


In [4]:
with gcs.open('soleadify_sample_data/unified_and_unmatched_data/unified_unique.csv') as f:
    unified_unique = pd.read_csv(f, quotechar='"', escapechar='\\')

In [5]:
unified_unique.describe(include='all')

,address_unified,category__facebook,category__google,category__website,city_clean_final,country_clean_final,description__facebook,description__google,email__facebook,index__facebook,index__google,index__website,language__website,legal_name__website,page_type__facebook,phone_unified,region_clean_final,root_domain_final,site_name_final,zip_code_unified
count,24373,20446,2834,23708,22290,24725,11161,3188,8517,26007.000000,3297.000000,23786.00000,22546,10444,26007,25407,22290,26345,26345,18297
unique,24265,5211,322,553,3369,77,11039,3186,8514,NaN,NaN,NaN,13,10073,4,25232,309,26344,26289,14847
top,"Surrey, BC, Canada",Auto Services,Construction Services,High Schools,Calgary,Canada,History Museum,No reviews · Accounting firm Closed ⋅ Opens 8A...,ford.can.gp@gmail.com,NaN,NaN,NaN,en,"Blackboard, Inc.",LocalBusiness,+611301000000,Ontario,johnnysmoving.com,Aquinas College,victoria
freq,12,344,110,513,1615,19370,106,2,4,NaN,NaN,NaN,20576,85,20604,22,8953,2,3,227
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35938.331411,176516.428571,36036.76873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20849.970334,100354.986163,20779.47816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,696.000000,3.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17871.500000,90874.000000,17944.50000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35887.000000,175392.000000,36016.50000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54009.000000,261491.000000,54102.75000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
